In [1]:
import os
import sys
sys.path.append('../')
os.chdir('../')

In [2]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
import duck_db_helper
import duckdb
from datetime import date

In [3]:
table_name = 'train_feature_1'
db = '../data/feature_generation/train.db'
conn = duckdb.connect('../data/feature_generation/train.db')
df = conn.sql(f"SELECT * FROM \"{table_name}\";").df()
conn.close()

In [4]:
model_formula = "total_points ~ name*(temperature+odds_for+last_3_matches_average)"
model = smf.ols(model_formula, data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           total_points   R-squared:                       0.320
Model:                            OLS   Adj. R-squared:                  0.129
Method:                 Least Squares   F-statistic:                     1.675
Date:                Thu, 11 Jan 2024   Prob (F-statistic):           6.69e-34
Time:                        13:02:50   Log-Likelihood:                -13438.
No. Observations:                5844   AIC:                         2.944e+04
Df Residuals:                    4560   BIC:                         3.801e+04
Df Model:                        1283                                         
Covariance Type:            nonrobust                                         
                                                                         coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

In [5]:
model_name = 'model1'
model.save(f"../data/models/{model_name}.pickle")
# create a database if it doesn't exist, that is the model registry. 
# name_of_model | dataset_used | date_trained
with open('../data/models/train_registry.csv', mode='a') as f:
    f.write(f'{str(date.today())},{model_name},{model_formula},{db}:{table_name},{model.rsquared}\n')